<a href="https://colab.research.google.com/github/alhasanmolla/Quant-Researcher/blob/main/Advanced_LBO_Model_for_Quantitative_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install numpy-financial

import numpy as np
import numpy_financial as npf # Import numpy_financial

class LBOModel:
    """
    Advanced LBO Model for Quantitative Analysis.

    Attributes:
        purchase_price (float): Enterprise value (in ₹ Crore).
        equity_contribution (float): Equity portion (in ₹ Crore).
        interest_rate (float): Annual interest rate on debt.
        revenue_base (float): Base revenue for year 0.
        revenue_growth (float): Annual revenue growth rate.
        ebitda_margin (float): EBITDA as a fraction of revenue.
        debt_repayment (float): Annual debt repayment (in ₹ Crore).
        exit_multiple (float): Exit multiple on final year EBITDA.
        years (int): Investment horizon (number of years).
    """
    def __init__(self, purchase_price, equity_contribution, interest_rate,
                 revenue_base, revenue_growth, ebitda_margin, debt_repayment,
                 exit_multiple, years):
        self.purchase_price = purchase_price
        self.equity_contribution = equity_contribution
        self.debt_raised = purchase_price - equity_contribution
        self.interest_rate = interest_rate
        self.revenue_base = revenue_base
        self.revenue_growth = revenue_growth
        self.ebitda_margin = ebitda_margin
        self.debt_repayment = debt_repayment
        self.exit_multiple = exit_multiple
        self.years = years

    def project_revenue(self):
        """Projects revenue for each year."""
        return [self.revenue_base * (1 + self.revenue_growth) ** i for i in range(self.years)]

    def project_ebitda(self, revenue):
        """Calculates EBITDA for each year based on revenue and margin."""
        return [rev * self.ebitda_margin for rev in revenue]

    def debt_schedule(self):
        """
        Computes the debt amortization schedule and interest payments.

        Returns:
            interest_list (list): Interest paid each year.
            remaining_debt_list (list): Remaining debt at the end of each year.
        """
        remaining_debt = self.debt_raised
        interest_list = []
        remaining_debt_list = []
        for i in range(self.years):
            interest = remaining_debt * self.interest_rate
            interest_list.append(interest)
            # Repay debt but ensure it does not go negative
            remaining_debt = max(0, remaining_debt - self.debt_repayment)
            remaining_debt_list.append(remaining_debt)
        return interest_list, remaining_debt_list

    def exit_valuation(self, ebitda, remaining_debt_list):
        """
        Computes the exit valuation and equity value at exit.

        Returns:
            exit_value (float): Total exit valuation based on final EBITDA.
            equity_value (float): Equity value at exit after paying off remaining debt.
        """
        final_ebitda = ebitda[-1]
        exit_value = final_ebitda * self.exit_multiple
        remaining_debt_end = remaining_debt_list[-1]
        equity_value = exit_value - remaining_debt_end
        return exit_value, equity_value

    def compute_irr(self, cash_flows):
        """
        Computes the Internal Rate of Return (IRR) given a cash flow series.

        Args:
            cash_flows (list): List of cash flows where the first element is the initial investment (negative).

        Returns:
            irr (float): Calculated IRR.
        """
        irr = npf.irr(cash_flows) # Use npf.irr instead of np.irr
        return irr

    def run_model(self):
        """
        Runs the complete LBO model simulation.

        Returns:
            dict: A dictionary containing projections, debt schedule, exit valuation, cash flows, and IRR.
        """
        # Revenue and EBITDA Projections
        revenue = self.project_revenue()
        ebitda = self.project_ebitda(revenue)

        # Debt schedule (interest payments and remaining debt)
        interest_list, debt_list = self.debt_schedule()

        # Exit valuation and final equity value
        exit_value, equity_value = self.exit_valuation(ebitda, debt_list)

        # Constructing the cash flow series
        # Start with the initial negative equity investment
        cash_flows = [-self.equity_contribution]
        # For each year, assume free cash flow is EBITDA minus interest and debt repayment
        for i in range(self.years):
            fcf = ebitda[i] - interest_list[i] - self.debt_repayment
            cash_flows.append(fcf)
        # In the final year, add the equity value from the exit
        cash_flows[-1] += equity_value

        irr = self.compute_irr(cash_flows)

        return {
            "Revenue": revenue,
            "EBITDA": ebitda,
            "Interest_Paid": interest_list,
            "Remaining_Debt": debt_list,
            "Exit_Value": exit_value,
            "Equity_Value_at_Exit": equity_value,
            "Cash_Flows": cash_flows,
            "IRR": irr
        }

# Example usage:
if __name__ == "__main__":
    # Define model parameters (values in ₹ Crore where applicable)
    model = LBOModel(
        purchase_price=500,        # Enterprise Value
        equity_contribution=100,   # Equity Investment
        interest_rate=0.08,        # 8% interest rate
        revenue_base=200,          # Base revenue in Year 0
        revenue_growth=0.07,       # 7% annual growth
        ebitda_margin=0.25,        # 25% EBITDA margin
        debt_repayment=50,         # Annual debt repayment
        exit_multiple=8,           # Exit multiple on EBITDA
        years=5                    # Investment horizon of 5 years
    )

    results = model.run_model()

    # # Displaying the results
    # for key, value in results.items():
    #     print("______________________________________")
    #     print("Advanced LBO Model for Quantitative Analysis")
    #     print(f"{key}: {value:.2f}")

    # Displaying the results
    for key, value in results.items():
        print("______________________________________")
        print("Advanced LBO Model for Quantitative Analysis")
        if isinstance(value, list):  # Check if value is a list
            print(f"{key}: {[f'{v:.2f}' for v in value]}")  # Format each item in the list
        else:
            print(f"{key}: {value:.2f}")  # Format value if not a list

______________________________________
Advanced LBO Model for Quantitative Analysis
Revenue: ['200.00', '214.00', '228.98', '245.01', '262.16']
______________________________________
Advanced LBO Model for Quantitative Analysis
EBITDA: ['50.00', '53.50', '57.25', '61.25', '65.54']
______________________________________
Advanced LBO Model for Quantitative Analysis
Interest_Paid: ['32.00', '28.00', '24.00', '20.00', '16.00']
______________________________________
Advanced LBO Model for Quantitative Analysis
Remaining_Debt: ['350.00', '300.00', '250.00', '200.00', '150.00']
______________________________________
Advanced LBO Model for Quantitative Analysis
Exit_Value: 524.32
______________________________________
Advanced LBO Model for Quantitative Analysis
Equity_Value_at_Exit: 374.32
______________________________________
Advanced LBO Model for Quantitative Analysis
Cash_Flows: ['-100.00', '-32.00', '-24.50', '-16.75', '-8.75', '373.86']
______________________________________
Advanced L